In [1]:
import pandas as pd
import MarineDNA as md
import seaborn as sn
import numpy as np

In [2]:
file1 = "../../../Data/Flyer2018_16S_table_counts.tsv"
asvs1 = pd.read_csv(file1, index_col=0, sep="\t")

In [3]:
# Return 3-d array of Beta alpha and beta parameters for a data frame of raw read counts
# Output dimensions are (parameters, ASVs, samples)
def betaParams(raw_counts):
    def colParams(col):
        return (col + 1, col.sum() - col + 1)
    return np.stack(
        [colParams(raw_counts.iloc[:, i]) for i in range(raw_counts.shape[1])], 
        axis = 2,
        dtype = np.dtype(np.int64, metadata = {"asvs": asvs1.index, "samples": asvs1.columns})
    )

In [4]:
p = betaParams(asvs1)
p

array([[[  553,  7416,  8750, ...,  2016,  1848,  1887],
        [  211,  1934,  2809, ...,  8727,  9331,  8853],
        [  146,  2090,  2531, ...,    23,    29,    45],
        ...,
        [    1,     1,     1, ...,     1,     1,     1],
        [    1,     1,     1, ...,     1,     1,     1],
        [    1,     1,     1, ...,     1,     1,     1]],

       [[ 3411, 33496, 54308, ..., 48232, 56372, 51754],
        [ 3753, 38978, 60249, ..., 41521, 48889, 44788],
        [ 3818, 38822, 60527, ..., 50225, 58191, 53596],
        ...,
        [ 3963, 40911, 63057, ..., 50247, 58219, 53640],
        [ 3963, 40911, 63057, ..., 50247, 58219, 53640],
        [ 3963, 40911, 63057, ..., 50247, 58219, 53640]]])

In [5]:
# Return random draw from Beta distribution of read percentages.
# Output is a data frame with rows = samples ad columns = ASVs
def ranRelPct_new(beta_params, asLogOdds = True):
    # function to draw from distribution and set percentages to sum to 1 for each sample
    def betaCol(col):
        beta_dist = np.random.beta(col[0, :], col[1, :])
        return beta_dist / beta_dist.sum()
    # pre-allocate result array (NOTE: transposes data structure)
    result = np.empty([beta_params.shape[2], beta_params.shape[1]])
    # draw from for each sample
    for i in range(result.shape[0]):
        result[i, :] = betaCol(beta_params[:, :, i])
    if asLogOdds:
        result = np.log(result / (1 - result))
    return pd.DataFrame(result, index = beta_params.dtype.metadata["samples"], columns = beta_params.dtype.metadata["asvs"])

In [6]:
%%time
lo_df = ranRelPct_new(p)
lo_df

CPU times: user 20.1 ms, sys: 1.99 ms, total: 22.1 ms
Wall time: 21.1 ms


ASV ID            495c1bd1608a1dad54d3e2824ce899ef  \
CN18Fc12_8_eDNA                          -2.482497   
CN18Fc19_5_eDNA                          -1.602445   
CN18Fc21_6_eDNA                          -1.871843   
CN18Fc22_6_eDNA                          -1.701029   
CN18Fc24_6_eDNA                          -1.882129   
...                                            ...   
CN18SESPkoa_SC42                         -3.059620   
CN18SESPkoa_SC44                         -5.904932   
CN18SESPkoa_SC45                         -3.225555   
CN18SESPkoa_SC47                         -3.460759   
CN18SESPkoa_SC49                         -3.383331   

ASV ID            a900b6678ce86851fb16bfafb87f3326  \
CN18Fc12_8_eDNA                          -3.372453   
CN18Fc19_5_eDNA                          -3.067253   
CN18Fc21_6_eDNA                          -3.128108   
CN18Fc22_6_eDNA                          -3.205109   
CN18Fc24_6_eDNA                          -3.469615   
...                                            ...   
CN18SESPkoa_SC42                         -1.614692   
CN18SESPkoa_SC44                         -4.688109   
CN18SESPkoa_SC45                         -1.625303   
CN18SESPkoa_SC47                         -1.717478   
CN18SESPkoa_SC49                         -1.676020   

ASV ID            c8e360969108fa2125a3d56eb4dad24f  \
CN18Fc12_8_eDNA                          -3.929155   
CN18Fc19_5_eDNA                          -2.970020   
CN18Fc21_6_eDNA                          -3.222800   
CN18Fc22_6_eDNA                          -3.186796   
CN18Fc24_6_eDNA                          -3.074475   
...                                            ...   
CN18SESPkoa_SC42                         -6.902313   
CN18SESPkoa_SC44                         -5.208801   
CN18SESPkoa_SC45                         -7.737947   
CN18SESPkoa_SC47                         -7.896068   
CN18SESPkoa_SC49                         -7.171805   

ASV ID            72143fd9e63fe40c1258948d2f0d79c3  \
CN18Fc12_8_eDNA                          -3.941854   
CN18Fc19_5_eDNA                          -3.134322   
CN18Fc21_6_eDNA                          -3.196497   
CN18Fc22_6_eDNA                          -3.174771   
CN18Fc24_6_eDNA                          -3.131952   
...                                            ...   
CN18SESPkoa_SC42                         -5.872030   
CN18SESPkoa_SC44                         -4.820635   
CN18SESPkoa_SC45                         -6.199121   
CN18SESPkoa_SC47                         -6.272470   
CN18SESPkoa_SC49                         -6.618611   

ASV ID            7b6b178fad5599c0e9a734e4fb09fd64  \
CN18Fc12_8_eDNA                          -3.675396   
CN18Fc19_5_eDNA                          -3.168373   
CN18Fc21_6_eDNA                          -3.584604   
CN18Fc22_6_eDNA                          -3.311810   
CN18Fc24_6_eDNA                          -3.344010   
...                                            ...   
CN18SESPkoa_SC42                         -6.436036   
CN18SESPkoa_SC44                         -4.680770   
CN18SESPkoa_SC45                         -6.867544   
CN18SESPkoa_SC47                         -6.637340   
CN18SESPkoa_SC49                         -7.065730   

ASV ID            4bbec3bb723375416616a87d785ac74a  \
CN18Fc12_8_eDNA                          -4.868287   
CN18Fc19_5_eDNA                          -4.489017   
CN18Fc21_6_eDNA                          -4.431885   
CN18Fc22_6_eDNA                          -4.661550   
CN18Fc24_6_eDNA                          -4.280186   
...                                            ...   
CN18SESPkoa_SC42                         -4.616366   
CN18SESPkoa_SC44                         -3.224438   
CN18SESPkoa_SC45                         -5.009891   
CN18SESPkoa_SC47                         -5.197082   
CN18SESPkoa_SC49                         -5.198362   

ASV ID            0c35cfa523aa27921ef8544a16d1cd36  \
CN18Fc12_8_eDNA                          -7.175893  